# Rossmann Store Sales

- Bases de dados: https://www.kaggle.com/c/rossmann-store-sales/data


O uso de Machine Learning é importante para aumentar a competitividade das empresas ao prever vendas futuras. Essas bases de dados contém históricos de vendas de 1.115 lojas Rossmann.

**O objetivo deste projeto é prever vendas diárias futuras. Nele será utilizado a ferramenta Facebook Prophet que é um framework do Facebook utilizado para previsão de vendas e valores numéricos no geral.**

Modelos de previsão de vendas baseados em dados passados devem considerar o que chamamos de efeitos sazonais, que são efeitos como demanda, feriados, concorrência e promoções.

As base de dados contém histórico de transações e dados das lojas:

- **id**- identificador da transação (loja+data)
- **Store** - identificador único para cada loja
- **Sales** - o volume de negócios/vendas para um determinado dia (objetivo)
- **Customers** - o número de clientes em um determinado dia
- **Open** - um indicador para saber se a loja estava aberta: 0 = fechado, 1 = aberto
- **StateHoliday** - indica um feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas fecham nos feriados e fins de semana. 
 - a = feriado, b = feriado da Páscoa, c = Natal, 0 = nenhum

- **SchoolHoliday** - feriado escolar
- **StoreType** - diferencia entre 4 modelos de loja diferentes: a, b, c, d
- **Assortment** - descreve um nível de sortimento: 
  - a = básico, b = extra, c = estendido
- **CompetitionDistance** - distância em metros até a loja concorrente mais próxima
- **CompetitionOpenSince[Month/Year]** - fornece o ano e o mês aproximados da hora em que o concorrente mais próximo foi aberto
- **Promo** - indica se uma loja está realizando uma promoção naquele dia
- **Promo2** - Promo2 é uma promoção contínua e consecutiva para algumas lojas: 
  - 0 = loja não está participando, 1 = loja está participando
- **Promo2Since[Year/Week]** - descreve o ano e a semana em que a loja começou a participar do Promo2
- **PromoInterval** - descreve os intervalos consecutivos em que o Promo2 é iniciado, nomeando os meses em que a promoção é reiniciada. Por exemplo. "Fevereiro, maio, agosto, novembro" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para essa loja



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
sales_train_df = pd.read_csv('./data/train.csv')

In [4]:
print(f'A base de dados contém {sales_train_df.shape[0]} linhas e {sales_train_df.shape[1]} colunas')

A base de dados contém 1017209 linhas e 9 colunas


In [5]:
sales_train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
for i in ['DayOfWeek','Open','Promo','SchoolHoliday']:
  print(f'{i} values: {sales_train_df[i].unique()}')

DayOfWeek values: [5 4 3 2 1 7 6]
Open values: [1 0]
Promo values: [1 0]
SchoolHoliday values: [1 0]


In [7]:
# dados das lojas
store_info_df = pd.read_csv('./data/store.csv')

In [8]:
print(f'A base de dados contém {store_info_df.shape[0]} linhas e {store_info_df.shape[1]} colunas')

A base de dados contém 1115 linhas e 10 colunas


In [9]:
store_info_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


### Unindo as tabelas

In [34]:
df_stores = pd.merge(sales_train_df, store_info_df, how='left', on='Store')

In [11]:
df_stores.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [12]:
df_stores['Date'].dtype

dtype('O')

In [35]:
# conversão da coluna Date de string para datetime
df_stores['Date'] = pd.to_datetime(df_stores['Date'])

In [14]:
df_stores.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

### Substituindo os dados faltantes

Uma das maneiras de lidar com dados faltantes seria pensar primeiro no negócio. Se a coluna `CompetitionDistance` contém valores vazios, isso significa que não há competidores próximos. Então uma maneira de substituir estes vazios seria colocando uma distância bem maior que a distância máxima, pois seria o mesmo que não ter um competidor próximo dado a alta distância.

In [15]:
from math import isnan

In [16]:
df_stores['CompetitionDistance'].max()

75860.0

In [36]:
df_stores['CompetitionDistance']  = df_stores['CompetitionDistance'].apply(lambda x: 200000.0 if isnan(x) else x)

Podem haver duas razões para a coluna `CompetitionOpenSinceMonth` estar vazia: 1 - a loja não tem um competidor mais próximo, logo, não uma data de abertura de competidor mais próxima; 2 - a loja tem um competidor próximo, mas não se sabe a data em que esse competidor abriu (ou talvez esqueceram de registrar).


Então para os valores faltantes, podemos pegar a data de venda da coluna `Date` e extrair o mês para copiar para o valor faltante na coluna `CompetitionOpenSinceMonth`.

In [22]:
df_stores.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
161470,911,1,2015-03-09,9876,909,1,0,0,0,a,c,16490.0,NaN,NaN,0,NaN,NaN,NaN


Isso vai partir da premissa de que se a coluna `CompetitionDistance` está preenchida existe um competidor, só não sabemos quando ele abriu ainda. Então se eu tenho uma venda no mês 02 e eu coloco esse mês no lugar do NaN na coluna `CompetitionOpenSinceMonth`, a partir daí para próximas vendas eu consigo ver o comportamento delas desde que esse competidor abriu.

In [37]:
df_stores['CompetitionOpenSinceMonth'] = df_stores.apply(
    lambda x: x['Date'].month if isnan(x['CompetitionOpenSinceMonth']) else x['CompetitionOpenSinceMonth'],axis=1)

In [38]:
# usando a mesma lógica com o ano
df_stores['CompetitionOpenSinceYear'] = df_stores.apply(
    lambda x: x['Date'].year if isnan(x['CompetitionOpenSinceYear']) else x['CompetitionOpenSinceYear'],axis=1)

In [39]:
# usando a mesma lógica com Promo2Week/Year
df_stores['Promo2SinceWeek'] = df_stores.apply(
    lambda x: x['Date'].week if isnan(x['Promo2SinceWeek']) else x['Promo2SinceWeek'],axis=1)
df_stores['Promo2SinceYear'] = df_stores.apply(
    lambda x: x['Date'].year if isnan(x['Promo2SinceYear']) else x['Promo2SinceYear'],axis=1)

A coluna `PromoInterval` descreve o intervalo de meses que a Promo2 ficou ativa. Podemos extrair os meses da coluna `Date` e fazer um map sobre a coluna `PromoInterval`. Se um determinado mês estiver contido dentro de algum intervalo registrado na coluna `PromoInterval`, então isso significa que naquela venda houve promoção.

In [40]:
# dicionário para o map
month_map = {
    1:'Jan',
    2:'Fev',
    3:'Mar',
    4:'Apr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Aug',
    9:'Sep',
    10:'Oct',
    11:'Nov',
    12:'Dec'
}

In [41]:
# onde há NaN em PromoInterval, colocaremos 0
df_stores['PromoInterval'].fillna(0,inplace=True)

In [42]:
# mapeando os nomes dos meses
df_stores['MonthMap'] = df_stores['Date'].dt.month.map(month_map)

In [43]:
df_stores[['PromoInterval','Date','MonthMap']].head() 

,PromoInterval,Date,MonthMap
0,0,2015-07-31,Jul
1,"Jan,Apr,Jul,Oct",2015-07-31,Jul
2,"Jan,Apr,Jul,Oct",2015-07-31,Jul
3,0,2015-07-31,Jul
4,0,2015-07-31,Jul


In [44]:
# agora verificar se um determinado mês existe em PromoInterval
df_stores['IsPromo'] = df_stores[['PromoInterval','MonthMap']].apply(
    lambda x: 0 if x['PromoInterval']==0 else 1 if x['MonthMap'] in x['PromoInterval'].split(',') else 0, axis=1)

In [45]:
df_stores[['PromoInterval','Date','MonthMap','IsPromo']].head()

,PromoInterval,Date,MonthMap,IsPromo
0,0,2015-07-31,Jul,0
1,"Jan,Apr,Jul,Oct",2015-07-31,Jul,1
2,"Jan,Apr,Jul,Oct",2015-07-31,Jul,1
3,0,2015-07-31,Jul,0
4,0,2015-07-31,Jul,0


In [32]:
df_stores.isna().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
MonthMap                     0
IsPromo                      0
dtype: int64

In [33]:
df_stores.to_csv('stores_cleaning.csv', index=False)